In [1]:
import tensorflow as tf
import numpy as np
import pprint

import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

from data import WN18
from model.metrics import evaluate_rank

In [2]:
class Config:
    seed = 21
    n_epochs = 10
    batch_size = 128
    embed_dim = 150

In [3]:
"""
e: entity
s: subject
p: predicate
o: object
"""

def read_triples(path):
    triples = []
    with open(path, 'rt') as f:
        for line in f.readlines():
            s, p, o = line.split()
            triples += [(s.strip(), p.strip(), o.strip())]
    return triples


def load_triple():
    WN18.download()
    triples_tr = read_triples('../data/WN18/wn18/train.txt')
    triples_va = read_triples('../data/WN18/wn18/valid.txt')
    triples_te = read_triples('../data/WN18/wn18/test.txt')
    
    triples_all = triples_tr + triples_va + triples_te
    
    return triples_all, triples_tr, triples_va, triples_te


def build_vocab(triples):
    params = {}
    
    e_set = {s for (s, p, o) in triples} | {o for (s, p, o) in triples}
    p_set = {p for (s, p, o) in triples}

    params['e_vocab_size'] = len(e_set)
    params['p_vocab_size'] = len(p_set)

    e2idx = {e: idx for idx, e in enumerate(sorted(e_set))}
    p2idx = {p: idx for idx, p in enumerate(sorted(p_set))}
    
    return e2idx, p2idx, params


def build_train_data(triples_tr, e2idx, p2idx):
    x_s = np.array([e2idx[s] for (s, p, o) in triples_tr], dtype=np.int32)
    x_p = np.array([p2idx[p] for (s, p, o) in triples_tr], dtype=np.int32)
    x_o = np.array([e2idx[o] for (s, p, o) in triples_tr], dtype=np.int32)

    x = {'s': x_s,
         'p': x_p,
         'o': x_o}
    y = np.ones([len(x_s)], dtype=np.float32)
    
    return x, y


def train_input_fn(triples_tr, e2idx, p2idx, random_state, params):
    x, y = build_train_data(triples_tr, e2idx, p2idx)
    s, p, o = x['s'], x['p'], x['o']
    
    s_ = random_state.choice(params['e_vocab_size'], s.shape)
    o_ = random_state.choice(params['e_vocab_size'], o.shape)
    
    x_ = {
        's': np.concatenate([s, s_, s]),
        'p': np.concatenate([p, p, p]),
        'o': np.concatenate([o, o, o_])}
    y_ = np.concatenate([y, np.zeros([2*len(y)], dtype=np.float32)])
    
    return tf.estimator.inputs.numpy_input_fn(x = x_,
                                              y = y_,
                                              batch_size = Config.batch_size,
                                              num_epochs = 1,
                                              shuffle = True)

In [4]:
def forward(features, params):
    e_embed = tf.get_variable('e_embed',
                              [params['e_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    p_embed = tf.get_variable('p_embed',
                              [params['p_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    
    s = tf.nn.embedding_lookup(e_embed, features['s'])
    p = tf.nn.embedding_lookup(p_embed, features['p'])
    o = tf.nn.embedding_lookup(e_embed, features['o'])
    
    logits = tf.reduce_sum(s*p*o, axis=1)
    
    return logits
    
    
def model_fn(features, labels, mode, params):
    logits = forward(features, params)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        tf.logging.info('\n'+pprint.pformat(tf.trainable_variables()))
        
        loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                                         labels=labels))
        
        train_op = tf.train.AdamOptimizer().minimize(
            loss_op, global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode = mode,
                                          loss = loss_op,
                                          train_op = train_op)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions = tf.sigmoid(logits))

In [5]:
random_state = np.random.RandomState(Config.seed)
triples_all, triples_tr, triples_va, triples_te = load_triple()
e2idx, p2idx, params = build_vocab(triples_all)

model = tf.estimator.Estimator(model_fn,
                               params = params)

for _ in range(Config.n_epochs):
    model.train(train_input_fn(triples_tr,
                               e2idx,
                               p2idx,
                               random_state,
                               params))
evaluate_rank(model,
              triples_va,
              triples_te,
              triples_all,
              e2idx,
              p2idx,
              params['e_vocab_size'],
              batch_size = 1000)

Files Already Downloaded
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpxfdx2s52', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11f28eef0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 150) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 150) dtype=float32_ref>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorfl

INFO:tensorflow:loss = 0.67320466, step = 6117 (3.033 sec)
INFO:tensorflow:global_step/sec: 26.9827
INFO:tensorflow:loss = 0.7149087, step = 6217 (3.707 sec)
INFO:tensorflow:global_step/sec: 29.5411
INFO:tensorflow:loss = 0.7255721, step = 6317 (3.384 sec)
INFO:tensorflow:global_step/sec: 36.3261
INFO:tensorflow:loss = 0.73488474, step = 6417 (2.753 sec)
INFO:tensorflow:global_step/sec: 34.0926
INFO:tensorflow:loss = 0.70109457, step = 6517 (2.933 sec)
INFO:tensorflow:global_step/sec: 35.5462
INFO:tensorflow:loss = 0.69141084, step = 6617 (2.813 sec)
INFO:tensorflow:Saving checkpoints for 6632 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpxfdx2s52/model.ckpt.
INFO:tensorflow:Loss for final step: 0.7125488.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 150) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 150) dtype=float32_ref>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorf

INFO:tensorflow:global_step/sec: 34.8064
INFO:tensorflow:loss = 0.0754605, step = 12749 (2.873 sec)
INFO:tensorflow:global_step/sec: 35.8144
INFO:tensorflow:loss = 0.058684386, step = 12849 (2.792 sec)
INFO:tensorflow:global_step/sec: 34.6126
INFO:tensorflow:loss = 0.05425366, step = 12949 (2.889 sec)
INFO:tensorflow:global_step/sec: 33.7198
INFO:tensorflow:loss = 0.044854306, step = 13049 (2.965 sec)
INFO:tensorflow:global_step/sec: 31.6111
INFO:tensorflow:loss = 0.06358506, step = 13149 (3.164 sec)
INFO:tensorflow:global_step/sec: 36.5716
INFO:tensorflow:loss = 0.053853218, step = 13249 (2.734 sec)
INFO:tensorflow:Saving checkpoints for 13264 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpxfdx2s52/model.ckpt.
INFO:tensorflow:Loss for final step: 0.07054832.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 150) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 150) dtype=float32_ref>]
INFO:tensorflow:Done calling model_fn.
INF

INFO:tensorflow:loss = 0.02875885, step = 19181 (2.728 sec)
INFO:tensorflow:global_step/sec: 37.1654
INFO:tensorflow:loss = 0.021939669, step = 19281 (2.691 sec)
INFO:tensorflow:global_step/sec: 36.5401
INFO:tensorflow:loss = 0.023612075, step = 19381 (2.737 sec)
INFO:tensorflow:global_step/sec: 37.2711
INFO:tensorflow:loss = 0.026916528, step = 19481 (2.683 sec)
INFO:tensorflow:global_step/sec: 37.028
INFO:tensorflow:loss = 0.02215524, step = 19581 (2.701 sec)
INFO:tensorflow:global_step/sec: 36.8567
INFO:tensorflow:loss = 0.01589438, step = 19681 (2.713 sec)
INFO:tensorflow:global_step/sec: 37.1035
INFO:tensorflow:loss = 0.11334425, step = 19781 (2.695 sec)
INFO:tensorflow:global_step/sec: 36.6587
INFO:tensorflow:loss = 0.050227866, step = 19881 (2.728 sec)
INFO:tensorflow:Saving checkpoints for 19896 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpxfdx2s52/model.ckpt.
INFO:tensorflow:Loss for final step: 0.04212141.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Va

INFO:tensorflow:loss = 0.00483851, step = 25613 (2.723 sec)
INFO:tensorflow:global_step/sec: 35.7832
INFO:tensorflow:loss = 0.0077848253, step = 25713 (2.795 sec)
INFO:tensorflow:global_step/sec: 37.0247
INFO:tensorflow:loss = 0.0028385045, step = 25813 (2.701 sec)
INFO:tensorflow:global_step/sec: 36.023
INFO:tensorflow:loss = 0.0023916003, step = 25913 (2.776 sec)
INFO:tensorflow:global_step/sec: 37.1561
INFO:tensorflow:loss = 0.0020532545, step = 26013 (2.692 sec)
INFO:tensorflow:global_step/sec: 35.7471
INFO:tensorflow:loss = 0.0017043671, step = 26113 (2.797 sec)
INFO:tensorflow:global_step/sec: 36.4826
INFO:tensorflow:loss = 0.002305703, step = 26213 (2.741 sec)
INFO:tensorflow:global_step/sec: 36.0639
INFO:tensorflow:loss = 0.0011865097, step = 26313 (2.773 sec)
INFO:tensorflow:global_step/sec: 32.8178
INFO:tensorflow:loss = 0.02344466, step = 26413 (3.047 sec)
INFO:tensorflow:global_step/sec: 36.397
INFO:tensorflow:loss = 0.008224772, step = 26513 (2.747 sec)
INFO:tensorflow:Sav

INFO:tensorflow:global_step/sec: 35.7859
INFO:tensorflow:loss = 0.005378646, step = 32045 (2.794 sec)
INFO:tensorflow:global_step/sec: 27.4899
INFO:tensorflow:loss = 0.0037244281, step = 32145 (3.638 sec)
INFO:tensorflow:global_step/sec: 33.402
INFO:tensorflow:loss = 0.012326207, step = 32245 (2.994 sec)
INFO:tensorflow:global_step/sec: 36.4193
INFO:tensorflow:loss = 0.0032763141, step = 32345 (2.746 sec)
INFO:tensorflow:global_step/sec: 36.9928
INFO:tensorflow:loss = 0.0020790603, step = 32445 (2.703 sec)
INFO:tensorflow:global_step/sec: 35.7974
INFO:tensorflow:loss = 0.0013316611, step = 32545 (2.794 sec)
INFO:tensorflow:global_step/sec: 33.9071
INFO:tensorflow:loss = 0.0010133577, step = 32645 (2.949 sec)
INFO:tensorflow:global_step/sec: 30.993
INFO:tensorflow:loss = 0.0016930837, step = 32745 (3.227 sec)
INFO:tensorflow:global_step/sec: 33.1434
INFO:tensorflow:loss = 0.0009341946, step = 32845 (3.017 sec)
INFO:tensorflow:global_step/sec: 35.5292
INFO:tensorflow:loss = 0.0005773639,